In [4]:
import pandas as pd
import numpy as np
import csv
from tqdm import tqdm
import pickle

In [2]:
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

# Cargar datos

Primero, cargamos el csv, las hash signatures y los tweets unicos encontrados

### K = 5

In [195]:
k = 5
hash_funcs = 200
b = 20
r = 10

filtered_data = pd.read_csv('filtered_tweets_2022_abril_junio.csv', index_col=0, quotechar='"')
reduced_data = filtered_data[filtered_data['text'].apply(len) >= k]

similar_items = set()
with open(f"similar_items_k={k}_hash_funcs={hash_funcs}_b={b}_r={r}.pkl", 'rb') as f:
    similar_items = pickle.load(f)

unique_tweets = []
with open(f"unique_tweets_k={k}.pkl", 'rb') as f:
    unique_tweets = pickle.load(f)

Primero, armamos un diccionario, donde la llave sera un tweet, y el valor sera una lista con todos los usuarios que tweetearon eso. Recordemos que hay varios tweets que fueron tweeteados por mas de 1 persona.

In [196]:
tweets = {}
tweets_per_user = {}

for row in reduced_data.itertuples():

    try:
        if row.screen_name in tweets[row.text]:
            continue
        else:
            tweets[row.text].append(row.screen_name)
    except KeyError:
        tweets[row.text] = [row.screen_name]
    
    try:
        tweets_per_user[row.screen_name] += 1
    except KeyError:
        tweets_per_user[row.screen_name] = 1

Ahora, obtenemos los usuarios que tweetearon tweets similares. Para esto, mantenemos un diccionario que como llave tendra una tupla de 2 usuarios $(u_{1}, u_{2})$, y como valor tendra la cantidad de tweets similares entre esos 2 usuarios.

Para lograr esto, iteramos sobre todos los items similares $(i_{1}, i_{2})$, y para cada uno, obtenemos los usuarios que tweetearon el primer o segundo item. Luego, para cada par de usuarios $(u_{1}, u_{2})$ tal que $u_{1}$ tweeteo $i_{1}$ y $u_{2}$ tweeteo $i_{2}$, sumamos 1 al contador de $(u_{1}, u_{2})$.

In [159]:
similar_tweets_count = {}

for first_id, second_id in tqdm(similar_items):

    first_tweet = unique_tweets[first_id]
    second_tweet = unique_tweets[second_id]

    first_users = tweets[first_tweet]
    second_users = tweets[second_tweet]

    for first_user in first_users:
        for second_user in second_users:
            if first_user != second_user:
                try:
                    similar_tweets_count[(first_user, second_user)] += 1
                except KeyError:
                    similar_tweets_count[(first_user, second_user)] = 1

100%|██████████| 4716440/4716440 [00:22<00:00, 209641.26it/s]


In [203]:
absolute_threshold = 5
similar_threshold = 0.3

similar_users_final = set()

for key, item in tqdm(similar_tweets_count.items()):
    first_user_tweets = tweets_per_user[key[0]]
    second_user_tweets = tweets_per_user[key[1]]

    if item >= similar_threshold * (first_user_tweets + second_user_tweets) and item >= absolute_threshold:
        if key[0] > key[1]:
            similar_users_final.add(key)
        else:
            similar_users_final.add((key[1], key[0]))

100%|██████████| 10614101/10614101 [00:05<00:00, 2109140.84it/s]


In [204]:
len(similar_users_final)

185

In [205]:
similar_users_final

{('AION01', '777SHEM'),
 ('ArkonteR', '777SHEM'),
 ('Cony_V', 'AlikaSukni'),
 ('Cotalvez', '777SHEM'),
 ('Daniiconsu', '777SHEM'),
 ('DaphneLack', '777SHEM'),
 ('Defenixacamale1', '777SHEM'),
 ('FFQuakeProject', '777SHEM'),
 ('FFQuakeProject', 'AION01'),
 ('FFQuakeProject', 'Cotalvez'),
 ('Fernand95382651', '777SHEM'),
 ('FranCarbone86', '777SHEM'),
 ('FranCarbone86', 'FFQuakeProject'),
 ('Francis38907479', 'Cooke_Jr'),
 ('Francis38907479', 'FlavioYanez'),
 ('GEOWASHJOR', '777SHEM'),
 ('GEOWASHJOR', 'FFQuakeProject'),
 ('IreneZaiferluck', '777SHEM'),
 ('JaimeMedinaAlva', 'AlikaSukni'),
 ('JaimeMedinaAlva', 'Cony_V'),
 ('JeanetteVillena', 'Dafrbr'),
 ('LAGRINGAMERCED1', '15_rosana'),
 ('LAGRINGAMERCED1', 'Adriveronica1'),
 ('LAGRINGAMERCED1', 'CelinaVillalb19'),
 ('LAGRINGAMERCED1', 'GildaSagula2'),
 ('LAGRINGAMERCED1', 'JorgeRMaidana'),
 ('LAGRINGAMERCED1', 'Juanreyes_8'),
 ('LMManitas', 'LAGRINGAMERCED1'),
 ('LuzTamu', 'Albert_Tonstein'),
 ('Marioacunabuin', 'Mariana_SanBK'),
 ('Marit

In [202]:
list(reduced_data[reduced_data['screen_name'] == '777SHEM'].text)

['RT @1ERLukkax: 1500 pag. @777SHEM @jeannettekaz @biobio @barbarabricenok @LibardoBuitrago @coteevans @bdelamaza @RinconSalfate @FFQuakeProj…',
 'RT @1ERLukkax: Polémico @777SHEM @jeannettekaz @biobio @barbarabricenok @LibardoBuitrago @coteevans @bdelamaza @RinconSalfate @FFQuakeProje…',
 'RT @1ERLukkax: Más fría @777SHEM @jeannettekaz @biobio @barbarabricenok @LibardoBuitrago @coteevans @bdelamaza @RinconSalfate @FFQuakeProje…',
 'RT @1ERLukkax: Analicen @777SHEM @jeannettekaz @biobio @barbarabricenok @LibardoBuitrago @coteevans @bdelamaza @RinconSalfate @FFQuakeProje…',
 'RT @1ERLukkax: Comentario @777SHEM @jeannettekaz @biobio @barbarabricenok @LibardoBuitrago @coteevans @bdelamaza @RinconSalfate @FFQuakePro…',
 'RT @1ERLukkax: Hilo @777SHEM @jeannettekaz @biobio @barbarabricenok @LibardoBuitrago @coteevans @bdelamaza @RinconSalfate @FFQuakeProject @…',
 'RT @1ERLukkax: IPC @777SHEM @jeannettekaz @biobio @barbarabricenok @LibardoBuitrago @coteevans @bdelamaza @RinconSalfate @FFQuak

In [198]:
list(reduced_data[reduced_data['screen_name'] == 'AlikaSukni'].text)

['RT @sietepropuestas: Estimada Convencional \n@IvannaOlivares5 \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden re…',
 'RT @sietepropuestas: Estimada Convencional \n@MarielaSerey \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden recha…',
 'RT @sietepropuestas: Estimada Convencional \n@PatyLabraB \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden rechaza…',
 'RT @sietepropuestas: Estimada Convencional \n@damabarca \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden rechazar…',
 'RT @sietepropuestas: Estimado Convencional \n@AlvaroJofre \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden rechaz…',
 'RT @sietepropuestas: Estimado Convencional \n@Jaime_Bassa \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden rechaz…',
 'RT @sietepropuestas: Estimado Convencional \n@Jorgeabarcaxv \n ¿Porqué más de 4.000 funcionarios y fiscales del Mini

### K = 6

In [186]:
k = 6
hash_funcs = 200
b = 20
r = 10

filtered_data = pd.read_csv('filtered_tweets_2022_abril_junio.csv', index_col=0, quotechar='"')
reduced_data = filtered_data[filtered_data['text'].apply(len) >= k]

similar_items = set()
with open(f"similar_items_k={k}_hash_funcs={hash_funcs}_b={b}_r={r}.pkl", 'rb') as f:
    similar_items = pickle.load(f)

unique_tweets = []
with open(f"unique_tweets_k={k}.pkl", 'rb') as f:
    unique_tweets = pickle.load(f)

In [187]:
tweets = {}
tweets_per_user = {}

for row in reduced_data.itertuples():

    try:
        if row.screen_name in tweets[row.text]:
            continue
        else:
            tweets[row.text].append(row.screen_name)
    except KeyError:
        tweets[row.text] = [row.screen_name]
    
    try:
        tweets_per_user[row.screen_name] += 1
    except KeyError:
        tweets_per_user[row.screen_name] = 1

In [188]:
similar_tweets_count = {}

for first_id, second_id in tqdm(similar_items):

    first_tweet = unique_tweets[first_id]
    second_tweet = unique_tweets[second_id]

    first_users = tweets[first_tweet]
    second_users = tweets[second_tweet]

    for first_user in first_users:
        for second_user in second_users:
            if first_user != second_user:
                try:
                    similar_tweets_count[(first_user, second_user)] += 1
                except KeyError:
                    similar_tweets_count[(first_user, second_user)] = 1

100%|██████████| 3334975/3334975 [00:17<00:00, 194152.86it/s]


In [189]:
absolute_threshold = 10
similar_threshold = 0.5

similar_users_final = set()

for key, item in tqdm(similar_tweets_count.items()):
    first_user_tweets = tweets_per_user[key[0]]
    second_user_tweets = tweets_per_user[key[1]]

    if item >= similar_threshold * (first_user_tweets + second_user_tweets) and item >= absolute_threshold:
        if key[0] > key[1]:
            similar_users_final.add(key)
        else:
            similar_users_final.add((key[1], key[0]))

100%|██████████| 10614101/10614101 [00:04<00:00, 2165446.88it/s]


In [190]:
len(similar_users_final)

84

In [191]:
similar_users_final

{('ArkonteR', '777SHEM'),
 ('Cony_V', 'AlikaSukni'),
 ('Cotalvez', '777SHEM'),
 ('Daniiconsu', '777SHEM'),
 ('FFQuakeProject', '777SHEM'),
 ('FFQuakeProject', 'Cotalvez'),
 ('FranCarbone86', '777SHEM'),
 ('FranCarbone86', 'FFQuakeProject'),
 ('GEOWASHJOR', '777SHEM'),
 ('IreneZaiferluck', '777SHEM'),
 ('JaimeMedinaAlva', 'AlikaSukni'),
 ('JaimeMedinaAlva', 'Cony_V'),
 ('JeanetteVillena', 'Dafrbr'),
 ('LAGRINGAMERCED1', '15_rosana'),
 ('Marioacunabuin', 'Mariana_SanBK'),
 ('Maritza11015251', 'AlikaSukni'),
 ('Maritza11015251', 'Cony_V'),
 ('Maritza11015251', 'JaimeMedinaAlva'),
 ('MauriPe94118894', 'FromAnAU'),
 ('MauriPe94118894', 'HorstWenzel7'),
 ('Maurici72819481', 'JeanetteVillena'),
 ('Nell_Matta', 'Luiscor197348'),
 ('PaupinozaP', 'FranciscoRiosA'),
 ('RON1613', 'LAGRINGAMERCED1'),
 ('Rigobs', 'Nell_Matta'),
 ('SEMIDIOSRAMSES', 'Nell_Matta'),
 ('TakuBartolo', '777SHEM'),
 ('TakuBartolo', 'FFQuakeProject'),
 ('Viento_Riagny', '777SHEM'),
 ('Viento_Riagny', 'FFQuakeProject'),
 ('_j

In [192]:
list(reduced_data[reduced_data['screen_name'] == 'JaimeMedinaAlva'].text)

['RT @alinekuppenheim: Nunca mejor explicado',
 'RT @InfinitaFM: 🔴 Abogado José Francisco García (@chechegarcia): "Hay sobre la mesa una constitución que no es chavista, ni bolivariana, ni…',
 'RT @sietepropuestas: Estimado Convencional \n@amorenoe \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden rechazar…',
 'RT @cgajardop: Estimado Sr. Ossandón: parta  por conocer las penas de los delitos. El homicidio comienza en 10 años y un día no en 5 años y…',
 'RT @lucialopezchile: Duda más que plausible la de @SquellaAgustin 😊 #emparejandolacancha https://t.co/tFCnqZiXhC']

In [193]:
list(reduced_data[reduced_data['screen_name'] == 'Cony_V'].text)

['RT @sietepropuestas: Estimada Convencional \n@CotaSanJuan \n ¿Porqué pedimos rechazar los artículos 6 al 9 del informe de reemplazo de la com…',
 'RT @sietepropuestas: Estimada Convencional \n@VillenaNarbona \n¿Porqué pedimos rechazar los artículos 6 al 9 del informe de reemplazo de la c…',
 'RT @sietepropuestas: Estimada Convencional \n@CaroDistrito1 \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden rech…',
 'RT @sietepropuestas: Estimada Convencional \n@CarolCBown \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden rechaza…',
 'RT @sietepropuestas: Estimada Convencional \n@DayyanaGonzalez \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden re…',
 'RT @sietepropuestas: Estimada Convencional \n@ErickaPortillaB \n ¿Porqué más de 4.000 funcionarios y fiscales del Ministerio Público piden re…',
 'RT @sietepropuestas: Estimada Convencional \n@JanisMeneses_D6 \n ¿Porqué más de 4.000 funcionarios y fiscales del Mi

# OTROS

In [ ]:
('LAGRINGAMERCED1', '15_rosana')
